In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
no_masks, masks = [], []

for image_name in os.listdir('dataset/with_mask'):
    img = cv2.imread('dataset/with_mask/' + image_name)
    img=cv2.resize(img, (100, 100))
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=img/255
    masks.append(img)
    
for image_name in os.listdir('dataset/without_mask'):
    img = cv2.imread('dataset/without_mask/' + image_name)
    img=cv2.resize(img, (100, 100))
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=img/255
    no_masks.append(img)

In [3]:
len(masks), len(no_masks)

(690, 686)

In [4]:
target = [1]*690 + [0]*686
features = masks + no_masks

In [5]:
target = to_categorical(np.asarray(target))

In [6]:
features = np.array(features)

In [7]:
features.shape

(1376, 100, 100)

In [8]:
features = np.reshape(features, (1376, 100, 100, 1))

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1100, 100, 100, 1), (276, 100, 100, 1), (1100, 2), (276, 2))

In [18]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout
    
model = Sequential()
model.add(Conv2D(200, (5, 5), input_shape=(100, 100, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(200, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(Adam(lr=0.001),loss="categorical_crossentropy",metrics=["accuracy"])

In [19]:
# dataGen=ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,shear_range=0.1,rotation_range=10)
# dataGen.fit(X_train)

# model.fit_generator(dataGen.flow(X_train,y_train,batch_size=20),epochs=20)

model.fit(X_train, y_train, epochs=4)

Epoch 1/4
1100/1100 [==============================] - 257s 233ms/step - loss: 0.8150 - accuracy: 0.5418
Epoch 2/4
1100/1100 [==============================] - 256s 233ms/step - loss: 0.6267 - accuracy: 0.6427
Epoch 3/4
1100/1100 [==============================] - 252s 229ms/step - loss: 0.5373 - accuracy: 0.7409
Epoch 4/4
1100/1100 [==============================] - 250s 227ms/step - loss: 0.3629 - accuracy: 0.8482


In [ ]:
def preprocessing(img):
    img = cv2.imread('dataset/with_mask/' + image_name)
    img=cv2.resize(img, (100, 100))
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=img/255
    return img

In [15]:
model.save('my_model')

In [21]:
cap=cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)
cap.set(10,180)

False

In [22]:
while True:
    message,imgOriginal=cap.read()
    img=np.asarray(imgOriginal)
    img=cv2.resize(img, (100, 100))
    img=preprocessing(img)
    img=img.reshape(1,100 , 100, 1)
    cv2.putText(imgOriginal,"Class",(20,35),cv2.FONT_HERSHEY_TRIPLEX,1,(23,120,255),2)
    cv2.putText(imgOriginal,"Probability",(20,75),cv2.FONT_HERSHEY_TRIPLEX,1,(23,120,255),2)
    
    result = model.predict(img)
    output_1 = ''
    if result[0][1] > result[0][0]:output_1 = 'Mask'
    else:output_1 = 'No Mask'
    output_2 = max(result[0][0], result[0][1])
    cv2.putText(imgOriginal,output_1,(120,35),cv2.FONT_HERSHEY_TRIPLEX,1,(23,120,255),2)
    cv2.putText(imgOriginal,str(output_2) + "%",(250,75),cv2.FONT_HERSHEY_TRIPLEX,1,(23,120,255),2)
    cv2.imshow("Result",imgOriginal)
    
        


KeyboardInterrupt: 

In [23]:
cv2.destroyAllWindows()
cap.release()

In [21]:
tmp = model.predict(X_test[0].reshape(1, 100, 100, 1))

In [23]:
tmp[0][0]

0.3589279

In [24]:
tmp[0][1]

0.6410721